### Load dataset

- `path:` list of paths of chours.
- `titles:` list of titles of songs.
- `data:` a list contains data for each chours.

In [ ]:
import pandas as pd
dataset = pd.read_csv('Data/chorusData.csv')
paths = list(dataset['choruspath'])
titles = list(dataset['Title'])
data = []

## Two methods to extract features.
### First Method (statistics)
    That include all statistics we extract from features (skew, min, max, std, mean, median, kurtosis)
*`parameters`*
- `list:` list of feature we extracted from chours.
- `feature:` feature name shich we use with librosa.
- `columns_name:` list of all features we extracted which is 518 stored them to use it as a columns of dataframe
- `data:` data that contain 518 value which for every statistics for every feature. 

return data to collect all feature for it in second method....

### Second Method (extract_features)
    The main method of extract feature
*`parameters`*
- `audio_path:` path of chours we want to extract features from it.
- `title:` title of chours

return data and columns_name

In [ ]:
import librosa
import numpy as np
from scipy.stats import skew, kurtosis

def statistics(list, feature, columns_name, data):
    i = 0
    for ele in list:
        _skew = skew(ele)
        columns_name.append(f'{feature}_kew_{i}')
        min = np.min(ele)
        columns_name.append(f'{feature}_min_{i}')
        max = np.max(ele)
        columns_name.append(f'{feature}_max_{i}')
        std = np.std(ele)
        columns_name.append(f'{feature}_std_{i}')
        mean = np.mean(ele)
        columns_name.append(f'{feature}_mean_{i}')
        median = np.median(ele)
        columns_name.append(f'{feature}_median_{i}')
        _kurtosis = kurtosis(ele)
        columns_name.append(f'{feature}_kurtosis_{i}')

        i += 1
        data.append(_skew) 
        data.append(min)
        data.append(max) 
        data.append(std) 
        data.append(mean) 
        data.append(median) 
        data.append(_kurtosis)
    
    return data

def extract_features(audio_path, title):

    data = []
    columns_name = ['title']
    data.append(title)
    x , sr = librosa.load(audio_path)

    chroma_stft = librosa.feature.chroma_stft(x, sr)
    statistics(chroma_stft, 'chroma_stft', columns_name, data)

    chroma_cqt = librosa.feature.chroma_cqt(x, sr)
    statistics(chroma_cqt, 'chroma_cqt', columns_name, data)

    chroma_cens = librosa.feature.chroma_cens(x, sr)
    statistics(chroma_cens, 'chroma_cens', columns_name, data)

    mfcc = librosa.feature.mfcc(x, sr)
    statistics(mfcc, 'mfcc', columns_name, data)
    
    rms = librosa.feature.rms(x, sr)
    statistics(rms, 'rms', columns_name, data)

    spectral_centroid = librosa.feature.spectral_centroid(x , sr)
    statistics(spectral_centroid, 'spectral_centroid', columns_name, data)

    spectral_bandwidth = librosa.feature.spectral_bandwidth(x , sr)
    statistics(spectral_bandwidth, 'spectral_bandwidth', columns_name, data)

    spectral_contrast = librosa.feature.spectral_contrast(x , sr)
    statistics(spectral_contrast, 'spectral_contrast', columns_name, data)
    
    spectral_rolloff = librosa.feature.spectral_rolloff(x , sr)
    statistics(spectral_rolloff, 'spectral_rolloff', columns_name, data)

    tonnetz = librosa.feature.tonnetz(x , sr)
    statistics(tonnetz, 'tonnetz', columns_name, data)

    zero_crossing_rate = librosa.feature.zero_crossing_rate(x , sr)
    statistics(zero_crossing_rate, 'zero_crossing_rate', columns_name, data)

    return data, columns_name


### Here we extract features
*`parameters`*
- `i:` itrator to loop in paths.
- `audio_path:` path of chorus.
- `d:` data which returned from extract_features.
- `cols:` colmuns_name which returned from extract_features.

`NOTE` It took about 30 minutes

In [ ]:
i = 0
while i < len(paths):
    audio_path = paths[i]
    d, cols = extract_features(audio_path, titles[i])
    data.append(d)
    print(f'The {i} song Done...')
    i += 1

In [ ]:
# At first time we should convert data.
newData = pd.DataFrame(data, columns=cols)
newData.to_csv('Data/newData.csv')


# Because I do it in several times, I use this code but I must use code above in the first time.
# newData = pd.read_csv('Data/NewData.csv')
# for row in data:
#     newData = newData.append(row, ignore_index = True)

### Now add newData to dataset and clean it.

In [ ]:
# It's shape is (751, 523) if we deleted (Artist, Title, Label, Path, choruspath) columns it will be (751, 518 as mentioned in document)
finaldata = pd.concat([dataset, newData], axis=1, join='inner')
print(finaldata.shape)
finaldata.head(5)

In [ ]:
newtitles = list(newData['title'])
lostdata = []  # list to Titles and titles with doesn't matches
for i in range(len(titles)):
    if titles[i] != newtitles[i]:
        lostdata.append(titles[i])

# to see if any lost data or not. if(just delete rows) else dataset is good
lostdata

In [ ]:
# delete nan rows and data which titles doesn't match
# dataset = dataset[dataset.Title.isin(lostdata) == False]
# dataset.dropna(inplace=True)

In [ ]:
# if we deleted rows just reset index after that delete title row
# dataset.reset_index()
del finaldata['title']

In [ ]:
# Save last data
finaldata.to_csv('Data\Final Data.csv', index=False)

In [ ]:
import pandas as pd
df = pd.read_csv('Data\Final Data.csv')
print(df.shape)
df.head()

(751, 523)


,Artist,Title,Label,Path,choruspath,chroma_stft_kew_0,chroma_stft_min_0,chroma_stft_max_0,chroma_stft_std_0,chroma_stft_mean_0,...,tonnetz_mean_5,tonnetz_median_5,tonnetz_kurtosis_5,zero_crossing_rate_kew_0,zero_crossing_rate_min_0,zero_crossing_rate_max_0,zero_crossing_rate_std_0,zero_crossing_rate_mean_0,zero_crossing_rate_median_0,zero_crossing_rate_kurtosis_0
0,The Weeknd,Blinding Lights,1,Songs/The Weeknd - Blinding Lights (Official V...,Songs/RepeatedPopularChorus/The Weeknd - Blind...,0.091806,0.049545,1.0,0.306971,0.570429,...,0.007217,-0.000351,-0.410349,-0.604349,0.022585,0.102132,0.017701,0.072071,0.074626,0.012681
1,Olivia Rodrigo,Good 4 U,1,Songs/Olivia Rodrigo - good 4 u (Official Vide...,Songs/RepeatedPopularChorus/Olivia Rodrigo - g...,1.025965,0.007265,1.0,0.274179,0.350277,...,-0.011851,-0.010293,0.099909,-0.233246,0.057460,0.229705,0.035304,0.147578,0.157302,-0.281870
2,Olivia Rodrigo,Drivers License,1,Songs/Olivia Rodrigo - drivers license (Offici...,Songs/RepeatedPopularChorus/Olivia Rodrigo - d...,1.395750,0.000648,1.0,0.307888,0.236975,...,0.001545,-0.009260,0.360879,0.742759,0.023492,0.178957,0.032571,0.077563,0.066463,-0.180427
3,Lil Nas X,Montero (Call Me By Your Name),1,Songs/Lil Nas X - MONTERO (Call Me By Your Nam...,Songs/RepeatedPopularChorus/Lil Nas X - MONTER...,0.926616,0.003467,1.0,0.242382,0.301311,...,-0.014834,-0.023444,0.456362,0.524002,0.051111,0.277596,0.045581,0.147997,0.142472,-0.344706
4,BTS,Butter,1,Songs/BTS (방탄소년단) 'Butter' Official MV.mp3,Songs/RepeatedPopularChorus/BTS (방탄소년단) 'Butte...,0.311976,0.037900,1.0,0.336300,0.517165,...,0.005743,0.005232,-0.153372,2.700198,0.046848,0.235737,0.030815,0.097298,0.090204,7.939578
